In [15]:
# Import dependencies
import subprocess
import json
import os
from dotenv import load_dotenv

# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

# Import constants.py and necessary functions from bit and web3
# YOUR CODE HERE

from constants import *
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import *
from web3 import Web3
from eth_account import Account 

w3 = Web3(Web3.HTTPProvider("http://127.0.0.1.8545"))
 
 
# Create a function called `derive_wallets'
def derive_wallets(mnemonic, coin, numderive):
    command = f'./derive -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {"eth", "btc-test", "btc"}
numderive = 3
    
# Setting the dictionarry
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']


print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(keys, indent=4, sort_keys=True))

# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    
eth_acc = priv_key_to_account(ETH,eth_PrivateKey)
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)


# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, recipient, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to":recipient, "value": amount}
        )
        return { 
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])    
    

# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin,account,recipient, amount):
    if coin == "eth": 
        tx_eth = create_tx(coin,account, recipient, amount)
        sign_tx = account.signTransaction(tx_eth)
        result = w3.eth.sendRawTransaction(sign_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        tx_btctest= create_tx(coin,account,recipient,amount)
        sign_tx = account.sign_transaction(tx_btctest)      
        return NetworkAPI.broadcast_tx_testnet(sign_tx)     

"0x5bb4c3a9c8e7cd6436c82a5728edcca725ca242c913b1c1578c8adbc55cc93bc"
"cMnFrJMSpnbYziWvNbZjLyqtfsZC1x4qAddu13sj5sByWBz4xbCa"
{
    "btc": [
        {
            "address": "14Hk5VXBRB7ZaSorvg3NWEnUNmjdX54YbR",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "L4UR6QHkQKEJznH2jYbmMXdFqwYcyy2S8KpuDvRgwsS8hRn6ZCbn",
            "pubkey": "02764b3865769ecebbe0f564886f62f5267303579722bc5a53fb4720704108a235",
            "pubkeyhash": "2412fdf255f60620b1cffaf7d8559ad769b1704b",
            "xprv": "xprvA4DA4aZboGYP5sW5w8ouxkjEmezX1T87eLX9f1UKgRZWQu3MhTbEwsrhe1vgjaRcokTP2vERtFLQQVJBc3o5CzhunPmD89U5SLzUbyU88ph",
            "xpub": "xpub6HCWU66Vde6gJMaZ3ALvKtfyKgq1Quqy1ZSkTPswEm6VHhNWEzuVVgBBVGDEW8NS2trdgdDycpfmyXB25vtYoNwrGTJ7zSzg49k7pozbVUr"
        },
        {
            "address": "16zT71nif5rx9NzfN8bvQ4qWqJwFHsKzhQ",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "L4NMh4F5xXfjF76kq97KbZGPRX933mSZNuCBtArJSs2xWr5P

In [11]:
create_tx(BTCTEST,btc_acc,"mrXKsNqYu8wNf8zLihKjytY1Rk3WJJ92KR", 0.00001)

'{"unspents":[{"amount":100000,"confirmations":1,"script":"76a91478bb64a4d70b96554304363847b1842e1e765e4888ac","txid":"e1952606e9545dfd28c5776a5c89017597be3bb65c162c526d6e57152cbf78a3","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mrXKsNqYu8wNf8zLihKjytY1Rk3WJJ92KR",1000],["mrXKsNqYu8wNf8zLihKjytY1Rk3WJJ92KR",75948]]}'

In [13]:
send_tx(BTCTEST,btc_acc,"mrXKsNqYu8wNf8zLihKjytY1Rk3WJJ92KR", 0.00001)

In [41]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0x5bb4c3a9c8e7cd6436c82a5728edcca725ca242c913b1c1578c8adbc55cc93bc"))
from web3.middleware import geth_poa_middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)


In [42]:
w3.isConnected()

False

In [43]:
w3.eth.getBalance("0x712146Eaabec7B154cD9234f0aEBA738e6D0AaCc")

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8545): Max retries exceeded with url: /0x5bb4c3a9c8e7cd6436c82a5728edcca725ca242c913b1c1578c8adbc55cc93bc (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb8400b2dd0>: Failed to establish a new connection: [Errno 61] Connection refused'))